<a href="https://colab.research.google.com/github/tendiek/unsupervised-predict-streamlit-template/blob/master/Unsupervised_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

import operator 
import heapq 

# Imported for our sanity
import warnings
warnings.filterwarnings('ignore')


In [7]:
sample_df = pd.read_csv("sample_submission.csv")
sample_df.head

<bound method NDFrame.head of                   Id  rating
0             1_2011     1.0
1             1_4144     1.0
2             1_5767     1.0
3             1_6711     1.0
4             1_7318     1.0
...              ...     ...
5000014  162541_4079     1.0
5000015  162541_4467     1.0
5000016  162541_4980     1.0
5000017  162541_5689     1.0
5000018  162541_7153     1.0

[5000019 rows x 2 columns]>

In [8]:
train_df = pd.read_csv("train.csv")
train_df.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1.518350e+09
1,106343,5,4.5,1.206239e+09
2,146790,5459,5.0,1.076216e+09
3,106362,32296,2.0,1.423043e+09
4,9041,366,3.0,8.333758e+08


In [10]:
test_df = pd.read_csv("test.csv")
test_df.head()

,userId,movieId
0,5,788
1,68,7438
2,336,40412
3,803,3822
4,547,903


In [24]:
movie_df = pd.read_csv("movies.csv")
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
train_df.shape

(3611488, 4)

In [12]:
train_df.drop('timestamp', inplace=True, axis=1)
train_df.head()

,userId,movieId,rating
0,5163,57669,4.0
1,106343,5,4.5
2,146790,5459,5.0
3,106362,32296,2.0
4,9041,366,3.0


In [13]:
sampled_train = train_df.sample(n=50000)
sampled_train.head()

In [36]:
comb_movie = pd.merge(sampled_train, movie_df, left_on="movieId", right_on="movieId", how="left")
comb_movie.head()

,userId,movieId,rating,title,genres
0,1446,6870,4.0,Mystic River (2003),Crime|Drama|Mystery
1,111330,16,4.0,Casino (1995),Crime|Drama
2,27496,1225,3.0,Amadeus (1984),Drama
3,113881,7076,3.5,Bullitt (1968),Action|Crime|Drama|Thriller
4,54277,260,4.5,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi


In [14]:
util_matrix = sampled_train.pivot_table(index=['userId'],columns=['movieId'],values='rating') 
util_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,204698,204878,205074,205205,205287,205389,205425,206135,206843,207235
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
util_matrix_norm = util_matrix.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
util_matrix_norm.fillna(0, inplace=True)
util_matrix_norm = util_matrix_norm.T
util_matrix_norm = util_matrix_norm.loc[:, (util_matrix_norm != 0).any(axis=0)]
util_matrix_sparse = sp.sparse.csr_matrix(util_matrix_norm.values)
util_matrix_sparse

<7691x7849 sparse matrix of type '<class 'numpy.float64'>'
	with 21289 stored elements in Compressed Sparse Row format>

In [16]:
util_matrix_norm

userId,3,13,75,166,187,278,302,314,319,368,...,162387,162445,162481,162482,162485,162495,162501,162508,162516,162519
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
206135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
user_similarity = cosine_similarity(util_matrix_sparse.T) 
user_sim_df = pd.DataFrame(user_similarity, 
                           index = util_matrix_norm.columns, 
                           columns = util_matrix_norm.columns)
user_sim_df[:5]

userId,3,13,75,166,187,278,302,314,319,368,...,162387,162445,162481,162482,162485,162495,162501,162508,162516,162519
userId,,,,,,,,,,,,,,,,,,,,,
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
187,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
def collab_generate_top_N_recommendations(reference_user, N=10, k=20): 
    if reference_user not in user_sim_df.columns:
        return comb_movie.groupby(['movieId', 'title'] ).mean().sort_values(by='rating',
                                        ascending=False).index[:N].to_list()
    
    sim_users = user_sim_df.sort_values(by=reference_user, ascending=False).index[1:k+1]
    favorite_user_items = []  
    most_common_favorites = {} 
    
    for i in sim_users:
        max_score = util_matrix_norm.loc[:, i].max()  
        favorite_user_items.append(util_matrix_norm[util_matrix_norm.loc[:, i]==max_score].index.tolist())
        
    for item_collection in range(len(favorite_user_items)):
        for item in favorite_user_items[item_collection]: 
            if item in most_common_favorites:
                most_common_favorites[item] += 1
            else:
                most_common_favorites[item] = 1

    sorted_list = sorted(most_common_favorites.items(), key=operator.itemgetter(1), reverse=True)[:N]
    top_N = [x[0] for x in sorted_list]
    return top_N  

In [46]:
collab_generate_top_N_recommendations(12)

[(7932, 'Dark Days (2000)'),
 (1490, 'B*A*P*S (1997)'),
 (97933, '5 Broken Cameras (2011)'),
 (136878, 'Paradh (2010)'),
 (26699, 'Close-Up (Nema-ye Nazdik) (1990)'),
 (8367, 'Time of the Wolf, The (Le temps du loup) (2003)'),
 (4428, 'Misfits, The (1961)'),
 (188269, 'The Big Exchange (1992)'),
 (4437, 'Suspiria (1977)'),
 (137008, "A Mother's Nightmare (2012)")]

In [76]:
def movie_rating(reference_movie, reference_user, k=20, threshold=0.0):
    sim_users = user_sim_df.sort_values(by=reference_user, ascending=False).index[1:k+1]
    user_values = user_sim_df.sort_values(by=reference_user, ascending=False).loc[:,reference_user].tolist()[1:k+1]
    rating_list = [] 
    weight_list = []
    
    for sim_idx, user_id in enumerate(sim_users):
        rating = util_matrix.loc[user_id, reference_movie]
        similarity = user_values[sim_idx]
        if (np.isnan(rating)) or (similarity < threshold):
            continue
        elif not np.isnan(rating):
            rating_list.append(rating*similarity)
            weight_list.append(similarity)
    try:
        predicted_rating = sum(rating_list)/sum(weight_list) 
    except ZeroDivisionError:  
        predicted_rating = np.mean(util_matrix[reference_movie])
    return predicted_rating

In [64]:
y = sampled_train['rating']

X = sampled_train.drop('rating', axis=1)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [66]:
knn=KNeighborsRegressor(metric='cosine', n_neighbors=10)

In [67]:
knn.fit(X,y)

KNeighborsRegressor(metric='cosine', n_neighbors=10)

In [69]:
y_predicted = knn.predict(X_test)


In [75]:

df_predictions = pd.DataFrame(y_predicted, columns=['rating'])
df_predictions ['Id'] = test_df['userId'] +"_", test_df['movieId']
#f['col2']+"_"+df['col1'].fillna("").astype(str)
df_predictions.columns = ['Id','rating']
df_predictions = df_predictions[['Id','rating']]
df_predictions

UFuncTypeError: ignored